## Retail Analytics - Counter Service Analyzer

This guide walks us through building a custom analyzer, by extending result analyzer base class for retail analytics. Specifically, this analyzer tracks counter service time, idle time and average service time.

In [ ]:
import time
import cv2
import numpy as np
from collections import deque
from sklearn.metrics.pairwise import cosine_similarity
from degirum_tools.result_analyzer_base import ResultAnalyzerBase
import degirum as dg
import degirum_tools
from collections import defaultdict

## Counter Service Analyzer
We extend ResultAnalyzerBase class to write our custom analyzer logic for tracking counter service time

In [ ]:
class CounterServiceAnalyzer(ResultAnalyzerBase):
    """
    Analyzer for tracking service time at counters

    Tracks:
    - Service start/end time for each person at each counter
    - Counter idle time
    - Average service time across all completed services
    """

    def __init__(self, counter_zones, config=None):
        """
        Args:
            counter_zones: dict of {counter_id: np.array([[x1,y1], [x2,y2], ...])}
                          or list of np.arrays (will be auto-numbered as counter_1, counter_2, etc.)
            config: Optional configuration dict
        """
        super().__init__()

        # Convert list to dict if necessary
        if isinstance(counter_zones, list):
            self.counter_zones = {
                f"counter_{i+1}": np.array(zone, np.int32)
                for i, zone in enumerate(counter_zones)
            }
        else:
            self.counter_zones = {
                k: np.array(v, np.int32) for k, v in counter_zones.items()
            }

        # Configuration
        default_config = {
            "person_confidence": 0.35,
            "max_disappeared": 30,
            "tracking_threshold_percent": 10.0,
            "min_service_time": 1.0,  # Minimum time to count as a service (seconds)
        }
        self.config = {**default_config, **(config or {})}

        # Tracking state
        self.tracked_people = {}  # {person_id: PersonTrack}
        self.next_person_id = 1

        # Counter state
        self.counter_states = {
            counter_id: {
                "current_person": None,
                "service_start_time": None,
                "total_idle_time": 0.0,
                "last_idle_start": None,
                "is_idle": True,
            }
            for counter_id in self.counter_zones.keys()
        }

        # Service history
        self.completed_services = []  # List of service records
        self.last_5_services = deque(maxlen=5)

        # Frame management
        self.frame_count = 0
        self.fps = 30.0
        self.start_time = None

    def reset(self):
        """Reset all tracking state"""
        self.tracked_people = {}
        self.next_person_id = 1
        self.completed_services = []
        self.last_5_services.clear()
        self.frame_count = 0
        self.start_time = None

        # Reset counter states
        for counter_id in self.counter_states:
            self.counter_states[counter_id] = {
                "current_person": None,
                "service_start_time": None,
                "total_idle_time": 0.0,
                "last_idle_start": None,
                "is_idle": True,
            }

    def analyze(self, inference_result):
        """Main processing function called by DeGirum for each frame"""
        timestamp = self.frame_count / self.fps
        self.frame_count += 1

        if self.start_time is None:
            self.start_time = timestamp

        # Extract person detections
        person_detections = [
            det for det in inference_result.results
            if det["label"].lower() == "person" and det["score"] >= self.config["person_confidence"]
        ]

        # Update tracking
        image_width = inference_result.image.shape[1] if hasattr(inference_result, 'image') else 640
        self._update_tracking(person_detections, timestamp, image_width)

        # Update counter states
        self._update_counter_states(timestamp)

        # Add stats to inference result
        stats = self.get_stats()
        stats["timestamp"] = timestamp
        stats["frame_count"] = self.frame_count

        stats_result = {
            "label": "counter_service_stats",
            "score": 1.0,
            "stats": stats
        }
        inference_result.results.append(stats_result)

        # Annotate frame
        self._draw_annotations(inference_result.image_overlay, timestamp)

        return inference_result

    def annotate(self, inference_result, image):
        """Draw overlays on video frame"""
        timestamp = self.frame_count / self.fps
        self._draw_annotations(image, timestamp)
        return image

    def _match_to_existing_track(self, center, image_width):
        """Match detection to existing tracked person"""
        if not self.tracked_people:
            return None

        threshold_distance = (self.config["tracking_threshold_percent"] / 100.0) * image_width

        best_match = None
        best_distance = float('inf')

        for person_id, track in self.tracked_people.items():
            if track["frames_disappeared"] > self.config["max_disappeared"]:
                continue

            track_center = track["center"]
            distance = abs(center[0] - track_center[0]) + abs(center[1] - track_center[1])

            if distance < threshold_distance and distance < best_distance:
                best_distance = distance
                best_match = person_id

        return best_match

    def _get_person_counter(self, center):
        """Determine which counter (if any) the person is at"""
        for counter_id, zone in self.counter_zones.items():
            if self._point_in_polygon(center, zone):
                return counter_id
        return None

    def _update_tracking(self, detections, timestamp, image_width):
        """Update tracking state for all detections"""

        # Mark all as not seen this frame
        for person_id in self.tracked_people:
            self.tracked_people[person_id]["frames_disappeared"] += 1

        # Process each detection
        for det in detections:
            bbox = det["bbox"]
            center = self._get_center(bbox)
            counter_id = self._get_person_counter(center)

            # Try to match with existing track
            person_id = self._match_to_existing_track(center, image_width)

            # Create new track if no match
            if person_id is None:
                person_id = self.next_person_id
                self.next_person_id += 1

                self.tracked_people[person_id] = {
                    "center": center,
                    "bbox": bbox,
                    "frames_disappeared": 0,
                    "current_counter": None,
                    "service_start_time": None,
                }

            # Update track
            track = self.tracked_people[person_id]
            prev_counter = track["current_counter"]
            track["center"] = center
            track["bbox"] = bbox
            track["frames_disappeared"] = 0
            track["current_counter"] = counter_id

            # Handle counter entry
            if counter_id and prev_counter != counter_id:
                # Person entered a counter
                counter_state = self.counter_states[counter_id]

                # End previous person's service if any
                if counter_state["current_person"] is not None:
                    self._end_service(counter_state["current_person"], counter_id, timestamp)

                # Start new service
                track["service_start_time"] = timestamp
                counter_state["current_person"] = person_id
                counter_state["service_start_time"] = timestamp
                counter_state["is_idle"] = False

                # Stop counting idle time
                if counter_state["last_idle_start"] is not None:
                    idle_duration = timestamp - counter_state["last_idle_start"]
                    counter_state["total_idle_time"] += idle_duration
                    counter_state["last_idle_start"] = None

            # Handle counter exit
            elif prev_counter and not counter_id:
                # Person left counter
                self._end_service(person_id, prev_counter, timestamp)

        # Remove disappeared tracks
        to_remove = [
            pid for pid, track in self.tracked_people.items()
            if track["frames_disappeared"] > self.config["max_disappeared"]
        ]

        for person_id in to_remove:
            # End service if person was at counter
            if self.tracked_people[person_id]["current_counter"]:
                self._end_service(
                    person_id,
                    self.tracked_people[person_id]["current_counter"],
                    timestamp
                )
            del self.tracked_people[person_id]

    def _end_service(self, person_id, counter_id, timestamp):
        """End a service session"""
        track = self.tracked_people.get(person_id)
        if not track or track["service_start_time"] is None:
            return

        service_time = timestamp - track["service_start_time"]

        # Only record if service time meets minimum threshold
        if service_time >= self.config["min_service_time"]:
            service_record = {
                "person_id": person_id,
                "counter_id": counter_id,
                "service_start_time": round(track["service_start_time"], 2),
                "service_end_time": round(timestamp, 2),
                "service_duration": round(service_time, 2),
            }

            self.completed_services.append(service_record)
            self.last_5_services.append(service_record)

        # Clear track service info
        track["service_start_time"] = None
        track["current_counter"] = None

        # Update counter state
        counter_state = self.counter_states[counter_id]
        counter_state["current_person"] = None
        counter_state["service_start_time"] = None
        counter_state["is_idle"] = True
        counter_state["last_idle_start"] = timestamp

    def _update_counter_states(self, timestamp):
        """Update idle times for all counters"""
        for counter_id, state in self.counter_states.items():
            if state["is_idle"] and state["last_idle_start"] is None:
                state["last_idle_start"] = timestamp

    def _get_center(self, bbox):
        """Get center point of bounding box"""
        x1, y1, x2, y2 = bbox
        return (int((x1 + x2) / 2), int((y1 + y2) / 2))

    def _point_in_polygon(self, point, polygon):
        """Check if point is inside polygon"""
        return cv2.pointPolygonTest(polygon, point, False) >= 0

    def _draw_annotations(self, frame, timestamp):
        """Draw counter zones and tracking info on frame"""

        # Draw counter zones
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

        for idx, (counter_id, zone) in enumerate(self.counter_zones.items()):
            color = colors[idx % len(colors)]
            cv2.polylines(frame, [zone], True, color, 2)

            # Draw counter label
            zone_center = np.mean(zone, axis=0).astype(int)
            cv2.putText(frame, counter_id.upper(),
                       tuple(zone_center - [40, 10]),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Draw counter status
            state = self.counter_states[counter_id]
            status = "IDLE" if state["is_idle"] else f"P{state['current_person']}"
            cv2.putText(frame, status,
                       tuple(zone_center - [20, -10]),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Draw tracked people
        for person_id, track in self.tracked_people.items():
            if track["frames_disappeared"] > 0:
                continue

            bbox = track["bbox"]
            x1, y1, x2, y2 = map(int, bbox)

            # Color based on counter status
            color = (0, 255, 255) if track["current_counter"] else (255, 0, 255)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Draw person ID and service time
            cv2.putText(frame, f"person-{person_id}", (x1 + 80, y1 - 25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

            if track["service_start_time"] is not None:
                service_time = timestamp - track["service_start_time"]
                cv2.putText(frame, f"ST: {service_time:.1f}s", (x1 + 80, y1 - 5),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    def get_stats(self):
        """
        Get current statistics

        Returns:
            dict with current services, counter states, and aggregate stats
        """
        current_services = []
        for person_id, track in self.tracked_people.items():
            if track["service_start_time"] is not None and track["frames_disappeared"] == 0:
                current_time = self.frame_count / self.fps
                current_services.append({
                    "person_id": person_id,
                    "counter_id": track["current_counter"],
                    "service_start_time": round(track["service_start_time"], 2),
                    "current_service_time": round(current_time - track["service_start_time"], 2),
                })

        # Calculate counter idle times
        current_time = self.frame_count / self.fps
        counter_idle_times = {}
        for counter_id, state in self.counter_states.items():
            total_idle = state["total_idle_time"]
            if state["is_idle"] and state["last_idle_start"] is not None:
                total_idle += (current_time - state["last_idle_start"])
            counter_idle_times[counter_id] = round(total_idle, 2)

        # Calculate average service time
        avg_service_time = (
            round(sum(s["service_duration"] for s in self.completed_services) / len(self.completed_services), 2)
            if self.completed_services else 0.0
        )

        return {
            "current_services": current_services,
            "counter_idle_times": counter_idle_times,
            "average_service_time": avg_service_time,
            "total_services_completed": len(self.completed_services),
            "last_5_services": list(self.last_5_services),
            "completed_services": self.completed_services,
        }

## Attach analyzer and run inference

In [ ]:
if __name__ == "__main__":

    # Configuration
    inference_host_address = "@local"  # or "@cloud"
    zoo_url = "degirum/hailo"
    device_type = "HAILORT/HAILO8L"
    model_name="yolov8n_relu6_person--640x640_quant_hailort_multidevice_1"
    video_source = 'video.mp4' #replace video source

    counter_zones = {
    "counter_1": np.array([[800, 100], [1250, 100], [1250, 350], [800, 350]], np.int32),
    "counter_2": np.array([[400, 100], [800, 100], [800, 350], [400, 350]], np.int32),
    }

    # Load person detection model
    detection_model = dg.load_model(
        model_name = model_name,
        inference_host_address=inference_host_address,
        zoo_url=zoo_url
    )

    detection_load_time = time.time() - detection_start

    # Create analyzer
    service_analyzer = CounterServiceAnalyzer(
      counter_zones=counter_zones,
      config={
          "person_confidence": 0.35,
          "tracking_threshold_percent": 10.0,
          "min_service_time": 1.0,  # Minimum 1 second to count as service
      }
    )

    # Attach analyzer
    degirum_tools.attach_analyzers(detection_model, [service_analyzer])

    # Run inference
    print("Starting inference...")

    with degirum_tools.Display("Wait Time Tracker") as display:
        for result in degirum_tools.predict_stream(detection_model, video_source):
            display.show(result)

## Print stats from detection results

In [ ]:
detection["stats"]